In [61]:
# Mounting Drive

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [102]:
# Program design:

# Needs to take in machine file, input string, and flags and return info about machine as well as if string was accepted/rejected
# Depending on outcome of string, print accept path, reject path, or max step limit (if step limit is exceeded)

# reading in aplus csv file
import csv
# os.chdir('/content/drive/My Drive/Theory of Computing/Project2')

# setting global max_steps variable
max_depth = 100 # UPDATE IF NEEDED
max_transitions = 100  # UPDATE IF NEEDED

def print_path(state_sequence, accepted_config):
    print("Path to accept:")
    config_path = []
    current_state = accepted_config

    # Build path by backtracking through the tree
    while current_state is not None:
        config_path.insert(0, current_state)
        current_state = state_sequence.get(current_state)

    # Print each configuration with more detail
    for step_num, (tape_left, current_state, tape_right) in enumerate(config_path):
        tape_config = f"{tape_left}[{current_state}]{tape_right}"
        print(f"Step {step_num}: {tape_config}")


# NTM Tracing
def NTM_Tracer(file, input_string, termination_flag):
    """
    NTM Tracer Implementation

    Inputs:
        file: name of machine file
        input_string: input string to be read by machine
        termination_flag: stops execution if total number of steps exceeds set value (max_steps variable)

    Outputs:
        none
    """

    # Reading in machine file
    with open(file) as machine_file:
        csv_reader = csv.reader(machine_file, delimiter=',')
        line_count = 0
        reject_state = ''
        transitions = {}  # transitions dictionary
        for row in csv_reader:
            if line_count == 0:
                machine_name = row[0]
                line_count += 1
            elif line_count == 1:
                num_states = len(row)
                states = row
                line_count += 1
            elif line_count == 4:
                start_state = row[0]
                line_count += 1
            elif line_count == 5:
                accept_state = row[0]
                line_count += 1
            elif line_count == 6:
                reject_state = row[0]
                line_count += 1
            else:
                line_count += 1

    with open(file) as machine_file:
        csv_reader = csv.reader(machine_file, delimiter=',')
        line_count = 0
        for row in csv_reader:
            line_count += 1
            if line_count > 7:  # now can add to transition dictionary
                state, char, next_state, write_char, direction = row
                key = (state, char)
                if key not in transitions:
                    transitions[key] = []
                transitions[key].append((next_state, write_char, direction))

    num_transitions = 0
    for key in transitions:
        num_transitions += len(transitions[key])

    # echo name of machine, depth of tree configurations, and total number of transitions simulated
    print('Machine Name: ' + machine_name)
    print('Number of States: ' + str(num_states))
    print('Start State: ' + start_state)
    print('Accept State: ' + accept_state)
    print('Reject State: ' + reject_state)

    # breadth-first exploration of NTM with given string as input (input_string)
    state_sequence = {}  # Tree keeping track of all levels of the machine
    initial_config = ("", start_state, input_string)
    current_configs = [initial_config]
    transition_count = 0
    transition_list = []
    determinism_metric = 0

    while current_configs and transition_count < max_transitions:
        next_configs = []

        for current_config in current_configs:
            tape_left, current_state, tape_right = current_config

            # Check accept/reject before generating next configurations
            if current_state == accept_state:
                print(f"String accepted at {transition_count} transitions")
                print_path(state_sequence, current_config)
                return True, determinism_metric

            if current_state == reject_state: # need to see ALL possible paths
                continue

            # Get current tape head character
            head_char = tape_right[0] if tape_right else "_"

            # Generate next configurations
            transition_key = (current_state, head_char)
            if transition_key in transitions:
              current_transitions = 0
              for next_state, write_char, move_direction in transitions[transition_key]:
                    new_left = tape_left
                    new_right = tape_right

                    if move_direction == "R":
                        new_left = tape_left + write_char
                        new_right = tape_right[1:] if len(tape_right) > 1 else "_"
                    elif move_direction == "L":
                        new_left = tape_left[:-1] if tape_left else "_"
                        new_right = (write_char + tape_right[1:]) if len(tape_right) > 1 else write_char

                    new_config = (new_left, next_state, new_right)
                    next_configs.append(new_config)

                    # Only add to tree if we haven't seen this configuration
                    if new_config not in state_sequence:
                        state_sequence[new_config] = current_config
                        transition_count += 1
                        current_transitions += 1

            transition_list.append(current_transitions)
            determinism_metric = sum(transition_list) / len(transition_list)

        # update the current level
        current_configs = next_configs

        # if max_steps is exceeded, return and alert
        if len(state_sequence) >= max_depth and termination_flag == 1:
            print(f"Execution stopped at max depth of {max_depth}")
            return False, determinism_metric

    # if all configurations lead to reject
    print(f"String rejected at {len(state_sequence)} transitions")
    return False, determinism_metric

# Getting path to test csv file
file_path = '/content/drive/My Drive/Theory of Computing/Project2/'
d_metric = 0

# CHANGE IF YOU WANT TO SWITCH BETWEEN TEST CASES
test_case = 2

if test_case == 1:
  file_name = 'aplus.csv'
  file_path += file_name
  # Input strings for aplus test case
  input_string1 = 'aaab'
  input_string2 = 'aaa'
  input_string3 = 'aaaaaaaaaaa'
  input_string4 = 'aaabaaaaaa'
  input_string5 = 'aaaaaa'

if test_case == 2:
  file_name = 'abc_star.csv'
  file_path += file_name
  # Input strings for abc star test case
  input_string1 = 'abbbbbbb'
  input_string2 = 'abc'
  input_string3 = 'aaabbbccc'
  input_string4 = 'abbbabbc'
  input_string5 = 'aabbccccccc'

print('Trace 1 (should reject)')
print('Input String: ' + input_string1)
result1, d_metric = NTM_Tracer(file_path, input_string1, 1)
print('Determinism metric: ' + str(d_metric))

print('\nTrace 2 (should accept)')
print('Input String: ' + input_string2)
result2, d_metric = NTM_Tracer(file_path, input_string2, 1)
print('Determinism metric: ' + str(d_metric))

print('\nTrace 3 (should accept)')
print('Input String: ' + input_string3)
result3, d_metric = NTM_Tracer(file_path, input_string3, 1)
print('Determinism metric: ' + str(d_metric))

print('\nTrace 4 (should reject)')
print('Input String: ' + input_string4)
result4, d_metric = NTM_Tracer(file_path, input_string4, 1)
print('Determinism metric: ' + str(d_metric))

print('\nTrace 5 (should accept)')
print('Input String: ' + input_string4)
result5, d_metric = NTM_Tracer(file_path, input_string5, 1)
print('Determinism metric: ' + str(d_metric))

print('\n')

Trace 1 (should reject)
Input String: abbbbbbb
Machine Name: a*b*c* Nondeterministic
Number of States: 6
Start State: q0
Accept State: qacc
Reject State: qrej
String accepted at 26 transitions
Path to accept:
Step 0: [q0]abbbbbbb
Step 1: a[q0]bbbbbbb
Step 2: ab[q1]bbbbbb
Step 3: abb[q1]bbbbb
Step 4: abbb[q1]bbbb
Step 5: abbbb[q1]bbb
Step 6: abbbbb[q1]bb
Step 7: abbbbbb[q1]b
Step 8: abbbbbbb[q3]_
Step 9: abbbbbbb_[qacc]_
Determinism metric: 1.3617021276595744

Trace 2 (should accept)
Input String: abc
Machine Name: a*b*c* Nondeterministic
Number of States: 6
Start State: q0
Accept State: qacc
Reject State: qrej
String accepted at 10 transitions
Path to accept:
Step 0: [q0]abc
Step 1: a[q0]bc
Step 2: ab[q2]c
Step 3: abc[q3]_
Step 4: abc_[qacc]_
Determinism metric: 1.0

Trace 3 (should accept)
Input String: aaabbbccc
Machine Name: a*b*c* Nondeterministic
Number of States: 6
Start State: q0
Accept State: qacc
Reject State: qrej
String accepted at 28 transitions
Path to accept:
Step 0: [q0]